In [1]:
import pandas_gbq as pdq
import pandas as pd
import pydata_google_auth
import win32com.client as win32
from datetime import datetime as dt, timedelta
date = dt.today().strftime('%m.%d.%Y')
namefile = 'Data Request ' + date + '.xlsx'
SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    'https://www.googleapis.com/auth/drive',
]

credentials = pydata_google_auth.get_user_credentials(
    SCOPES,
    # Set auth_local_webserver to True to have a slightly more convienient
    # authorization flow. Note, this doesn't work if you're running from a
    # notebook on a remote sever, such as over SSH or with Google Colab.
    auth_local_webserver=True,
)

project = 'nbcu-ds-sandbox-a-001'

In [2]:
Max_Date = pdq.read_gbq("""SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE Territory IN('GB', 'IE')""",
project_id=project,
credentials=credentials)

Min_Date = pdq.read_gbq("""SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 30
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE Territory IN('GB', 'IE')""",
project_id=project,
credentials=credentials)

date_range = Min_Date.iloc[0,0].strftime("%m/%d") + ' - ' + Max_Date.iloc[0,0].strftime("%m/%d")
date_range

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.10rows/s]


'04/28 - 05/27'

In [3]:
MAAs_Territory_Provider = pdq.read_gbq(
    """
    -- Find the last date in the STB table to use to find trailing 30 days
    CREATE TEMP FUNCTION STB_Max_Date()
    AS(
        (SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE Territory IN('GB', 'IE'))
    );

    WITH SkyTV AS(
        SELECT
            "STB" AS Provider,
            Territory,
            PARSE_DATE('%Y%m%d', pdl_event_date) AS Watch_Date,
            customer_id AS ProfileId,
            '' AS ContentTitle
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE customer_id NOT IN(SELECT customer_id FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw` WHERE Territory = 'IT' AND pdl_event_date < '20220214')
    ),

    OTT AS(
        SELECT 
            "OTT" AS Provider,
            ProviderTerritory AS Territory,
            DATE(streamingSessionStart) AS Watch_Date,
            ProfileId,
            ContentTitle
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_ott_raw`
        WHERE ProgrammeViewingDuration >= 120
    ),

    Combined AS(
        SELECT * FROM OTT 
        UNION ALL 
        SELECT * FROM SkyTV 
    ),

    Intl AS(
        SELECT * EXCEPT(Territory), CASE WHEN Territory IN('GB', 'IE') THEN 'UK' WHEN Territory IN('DE', 'AT') THEN 'DE' WHEN Territory IN('IT') THEN 'IT' END Territory
        FROM Combined
        WHERE Combined.Watch_Date BETWEEN STB_Max_Date() - 29 AND STB_Max_Date() --AND Territory NOT IN('IT')
    )


    SELECT 
        Provider,
        Territory,
        COUNT(DISTINCT(ProfileId)) AS MAAs,
        --ROUND(COUNT(DISTINCT(ProfileId))/1000,0) AS MAAs_Rounded,
        --MIN(Watch_Date) AS Min_Date,
        --MAX(Watch_Date) AS Max_Date
    FROM Intl
    GROUP BY Provider, Territory
    -- ORDER BY Provider DESC, Territory DESC
    UNION ALL
    (SELECT 
        Provider,
        'Total' Territory,
        COUNT(DISTINCT(ProfileId)) AS MAAs,
        --ROUND(COUNT(DISTINCT(ProfileId))/1000,0) AS MAAs_Rounded,
        --MIN(Watch_Date) AS Min_Date,
        --MAX(Watch_Date) AS Max_Date
    FROM Intl
    GROUP BY Provider, Territory)
    UNION ALL
    (SELECT 
        'Total' Provider,
        'Total' Territory,
        COUNT(DISTINCT(ProfileId)) AS MAAs,
        --ROUND(COUNT(DISTINCT(ProfileId))/1000,0) AS MAAs_Rounded,
        --MIN(Watch_Date) AS Min_Date,
        --MAX(Watch_Date) AS Max_Date
    FROM Intl
    GROUP BY Provider, Territory)
    ORDER BY Provider, MAAs, Territory
    """,
project_id=project,
credentials=credentials)

Downloading: 100%|█████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 25.43rows/s]


In [4]:
Daily_AAs_STB = pdq.read_gbq(
    """
    -- Find the last date in the STB table to use to find trailing 30 days
    CREATE TEMP FUNCTION STB_Max_Date()
    AS(
        (SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE Territory IN('GB', 'IE'))
    );

    SELECT Watch_Date, Territory, COUNT(DISTINCT(customer_id)) MAAs
    FROM(
        SELECT PARSE_DATE('%Y%m%d', pdl_event_date) Watch_Date, 
        CASE 
            WHEN Territory IN('GB', 'IE') THEN 'UK'
            WHEN Territory IN('DE', 'AT') THEN 'DE'
            WHEN Territory IN('IT') THEN 'IT'
        END Territory,
        * EXCEPT(Territory)
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
    )
    WHERE Watch_Date <= STB_Max_Date()
    GROUP BY Watch_Date, Territory
    ORDER BY Watch_Date ASC, Territory ASC
    """,
project_id=project,
credentials=credentials)

Downloading: 100%|████████████████████████████████████████████████████████████████| 423/423 [00:00<00:00, 830.42rows/s]


In [5]:
Daily_AAs_OTT = pdq.read_gbq(
    """
    -- Find the last date in the STB table to use to find trailing 30 days
    CREATE TEMP FUNCTION STB_Max_Date()
    AS(
        (SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE Territory IN('GB', 'IE'))
    );

    SELECT Watch_Date, Territory, COUNT(DISTINCT(ProfileId)) MAAs
    FROM(
        SELECT DATE(streamingSessionStart) Watch_Date,
        CASE 
            WHEN ProviderTerritory IN('GB', 'IE') THEN 'UK'
            WHEN ProviderTerritory IN('DE', 'AT') THEN 'DE'
            WHEN ProviderTerritory IN('IT') THEN 'IT'
        END Territory,
        *
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_ott_raw`
        WHERE ProgrammeViewingDuration >= 120
    )
    WHERE Watch_Date <= STB_Max_Date()
    GROUP BY Watch_Date, Territory
    ORDER BY Watch_Date ASC, Territory
    """
)

Downloading: 100%|███████████████████████████████████████████████████████████████| 424/424 [00:00<00:00, 1136.08rows/s]


In [6]:
Daily_AAs_Starts_Usage = pdq.read_gbq(
    """
    -- Find the last date in the STB table to use to find trailing 30 days
    CREATE TEMP FUNCTION STB_Max_Date()
    AS(
        (SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE Territory IN('GB', 'IE'))
    );

    SELECT 
        Territory,
        Watch_Date,
        ContentTitle,
        COUNT(DISTINCT(ProfileID)) AAs,
        COUNT(ProfileID) ContentStarts,
        ROUND(SUM(ProgrammeViewingDuration)/3600, 1) Usage
    FROM `nbcu-ds-sandbox-a-001.wes_sandbox.OTT_With_Titles` 
    WHERE 
        ProgrammeViewingDuration >= 120 
        AND Watch_Date <=  STB_Max_Date()
    GROUP BY 1,2,3
    ORDER BY 1 DESC,2,3
    """,
project_id=project,
credentials=credentials)

Daily_AAs_Starts_Usage['Usage'] = Daily_AAs_Starts_Usage['Usage'].astype(float)

Downloading: 100%|██████████████████████████████████████████████████████████| 45215/45215 [00:04<00:00, 11180.13rows/s]


In [7]:
UK_First_7 = pdq.read_gbq(
    """
    -- Find the last date in the STB table to use to find trailing 30 days
    CREATE TEMP FUNCTION STB_Max_Date()
    AS(
        (SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE Territory IN('GB', 'IE'))
    );

    WITH Setup AS(
        SELECT *, MIN(Watch_Date) OVER(PARTITION BY ContentTitle) Launch_Date
        FROM `nbcu-ds-sandbox-a-001.wes_sandbox.OTT_With_Titles`
        WHERE 
            ProgrammeViewingDuration >= 120
            AND DATE(streamingSessionStart) <= STB_Max_Date()
            AND Territory = 'UK'
    )

    ,FIRST_7 AS(
        SELECT 
            ContentTitle,
            COUNT(DISTINCT(ProfileID)) First_7
        FROM Setup 
        WHERE Watch_Date BETWEEN Launch_Date AND Launch_Date + 6 
        GROUP BY ContentTitle
    )


    ,FIRST_30 AS(
        SELECT 
            ContentTitle,
            COUNT(DISTINCT(ProfileID)) First_30
        FROM Setup 
        WHERE Watch_Date BETWEEN Launch_Date AND Launch_Date + 29 
        GROUP BY ContentTitle
    )

    ,Latest_30 AS(
        SELECT 
            ContentTitle,
            COUNT(DISTINCT(ProfileID)) Latest_30
        FROM Setup 
        WHERE Watch_Date BETWEEN STB_Max_Date() - 29 AND STB_Max_Date() 
        GROUP BY ContentTitle
    )


    ,Launch_To_Date AS(
        SELECT 
            ContentTitle,
            Launch_Date,
            COUNT(DISTINCT(ProfileID)) Total_To_Date
        FROM Setup 
        GROUP BY ContentTitle, Launch_Date
    )

    SELECT 
        'UK' Territory,
        t.ContentTitle,
        t.Launch_Date,
        f7.First_7,
        f30.First_30,
        l30.Latest_30,
        t.Total_To_Date
    FROM Launch_To_Date t
    LEFT JOIN First_7 f7 ON t.ContentTitle = f7.ContentTitle
    LEFT JOIN First_30 f30 ON t.ContentTitle = f30.ContentTitle
    LEFT JOIN Latest_30 l30 ON t.ContentTitle = l30.ContentTitle
    ORDER BY ContentTitle
    """,
project_id=project,
credentials=credentials)

Downloading: 100%|████████████████████████████████████████████████████████████████| 233/233 [00:00<00:00, 651.53rows/s]


In [8]:
DE_First_7 = pdq.read_gbq(
    """
    -- Find the last date in the STB table to use to find trailing 30 days
    CREATE TEMP FUNCTION STB_Max_Date()
    AS(
        (SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE Territory IN('GB', 'IE'))
    );

    WITH Setup AS(
        SELECT *, MIN(Watch_Date) OVER(PARTITION BY ContentTitle) Launch_Date
        FROM `nbcu-ds-sandbox-a-001.wes_sandbox.OTT_With_Titles`
        WHERE 
            ProgrammeViewingDuration >= 120
            AND DATE(streamingSessionStart) <= STB_Max_Date()
            AND Territory = 'DE'
    )

    ,FIRST_7 AS(
        SELECT 
            ContentTitle,
            COUNT(DISTINCT(ProfileID)) First_7
        FROM Setup 
        WHERE Watch_Date BETWEEN Launch_Date AND Launch_Date + 6 
        GROUP BY ContentTitle
    )


    ,FIRST_30 AS(
        SELECT 
            ContentTitle,
            COUNT(DISTINCT(ProfileID)) First_30
        FROM Setup 
        WHERE Watch_Date BETWEEN Launch_Date AND Launch_Date + 29 
        GROUP BY ContentTitle
    )

    ,Latest_30 AS(
        SELECT 
            ContentTitle,
            COUNT(DISTINCT(ProfileID)) Latest_30
        FROM Setup 
        WHERE Watch_Date BETWEEN STB_Max_Date() - 29 AND STB_Max_Date() 
        GROUP BY ContentTitle
    )


    ,Launch_To_Date AS(
        SELECT 
            ContentTitle,
            Launch_Date,
            COUNT(DISTINCT(ProfileID)) Total_To_Date
        FROM Setup 
        GROUP BY ContentTitle, Launch_Date
    )

    SELECT 
        'DE' Territory,
        t.ContentTitle,
        t.Launch_Date,
        f7.First_7,
        f30.First_30,
        l30.Latest_30,
        t.Total_To_Date
    FROM Launch_To_Date t
    LEFT JOIN First_7 f7 ON t.ContentTitle = f7.ContentTitle
    LEFT JOIN First_30 f30 ON t.ContentTitle = f30.ContentTitle
    LEFT JOIN Latest_30 l30 ON t.ContentTitle = l30.ContentTitle
    ORDER BY ContentTitle
    """,
project_id=project,
credentials=credentials)

Downloading: 100%|████████████████████████████████████████████████████████████████| 128/128 [00:00<00:00, 374.62rows/s]


In [9]:
IT_First_7 = pdq.read_gbq(
    """
    -- Find the last date in the STB table to use to find trailing 30 days
    CREATE TEMP FUNCTION STB_Max_Date()
    AS(
        (SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE Territory IN('GB', 'IE'))
    );

    WITH Setup AS(
        SELECT *, MIN(Watch_Date) OVER(PARTITION BY ContentTitle) Launch_Date
        FROM `nbcu-ds-sandbox-a-001.wes_sandbox.OTT_With_Titles`
        WHERE 
            ProgrammeViewingDuration >= 120
            AND DATE(streamingSessionStart) <= STB_Max_Date()
            AND Territory = 'IT'
    )

    ,FIRST_7 AS(
        SELECT 
            ContentTitle,
            COUNT(DISTINCT(ProfileID)) First_7
        FROM Setup 
        WHERE Watch_Date BETWEEN Launch_Date AND Launch_Date + 6 
        GROUP BY ContentTitle
    )


    ,FIRST_30 AS(
        SELECT 
            ContentTitle,
            COUNT(DISTINCT(ProfileID)) First_30
        FROM Setup 
        WHERE Watch_Date BETWEEN Launch_Date AND Launch_Date + 29 
        GROUP BY ContentTitle
    )

    ,Latest_30 AS(
        SELECT 
            ContentTitle,
            COUNT(DISTINCT(ProfileID)) Latest_30
        FROM Setup 
        WHERE Watch_Date BETWEEN STB_Max_Date() - 29 AND STB_Max_Date() 
        GROUP BY ContentTitle
    )


    ,Launch_To_Date AS(
        SELECT 
            ContentTitle,
            Launch_Date,
            COUNT(DISTINCT(ProfileID)) Total_To_Date
        FROM Setup 
        GROUP BY ContentTitle, Launch_Date
    )

    SELECT 
        'IT' Territory,
        t.ContentTitle,
        t.Launch_Date,
        f7.First_7,
        f30.First_30,
        l30.Latest_30,
        t.Total_To_Date
    FROM Launch_To_Date t
    LEFT JOIN First_7 f7 ON t.ContentTitle = f7.ContentTitle
    LEFT JOIN First_30 f30 ON t.ContentTitle = f30.ContentTitle
    LEFT JOIN Latest_30 l30 ON t.ContentTitle = l30.ContentTitle
    ORDER BY ContentTitle
    """,
project_id=project,
credentials=credentials)

Downloading: 100%|████████████████████████████████████████████████████████████████| 104/104 [00:00<00:00, 291.64rows/s]


In [13]:
First7_Analysis = pd.concat([UK_First_7, DE_First_7, IT_First_7], ignore_index = True)

In [14]:
OTT_MAAs_Days_Watched = pdq.read_gbq(
    """
    -- Find the last date in the STB table to use to find trailing 30 days
    CREATE TEMP FUNCTION STB_Max_Date()
    AS(
        (SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE Territory IN('GB', 'IE'))
    );

    WITH Setup AS(
        SELECT 
            Days_Watched,
            Territory,
            COUNT(DISTINCT(ProfileID)) MAAs
        FROM(
            SELECT 
                *,
                CASE WHEN ProviderTerritory IN('GB', 'IE') THEN 'UK' WHEN ProviderTerritory IN('DE', 'AT') THEN 'DE' WHEN ProviderTerritory IN('IT') THEN 'IT' END Territory,
                COUNT(DISTINCT(Watch_Date)) OVER(PARTITION BY ProfileID) Days_Watched
            FROM(
                SELECT 
                    *, 
                    DATE(streamingSessionStart) Watch_Date,
                FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_ott_raw`
                WHERE 
                    ProgrammeViewingDuration >= 120
                    AND DATE(streamingSessionStart) BETWEEN STB_Max_Date() - 29 AND STB_Max_Date()
            )
        )
        GROUP BY Days_Watched, Territory
        ORDER BY Days_Watched
    )

    SELECT 
        Days_Watched, UK, DE, IT
    FROM(
        SELECT *
        FROM Setup
    ) AS p 
    PIVOT(
        SUM(MAAs) FOR Territory IN('UK', 'DE', 'IT')
    ) p2
    ORDER BY Days_Watched
    """,
project_id=project,
credentials=credentials)

Downloading: 100%|███████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 92.29rows/s]


In [15]:
OTT_MAAs_Titles_Watched = pdq.read_gbq(
    """
    -- Find the last date in the STB table to use to find trailing 30 days
    CREATE TEMP FUNCTION STB_Max_Date()
    AS(
        (SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE Territory IN('GB', 'IE'))
    );

    WITH Setup AS(
        SELECT 
            Titles_Watched,
            Territory,
            COUNT(DISTINCT(ProfileID)) MAAs
        FROM(
            SELECT 
                *,
                CASE WHEN ProviderTerritory IN('GB', 'IE') THEN 'UK' WHEN ProviderTerritory IN('DE', 'AT') THEN 'DE' WHEN ProviderTerritory IN('IT') THEN 'IT' END Territory,
                COUNT(DISTINCT(ContentTitle)) OVER(PARTITION BY ProfileID) Titles_Watched
            FROM(
                SELECT 
                    *, 
                    DATE(streamingSessionStart) Watch_Date,
                FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_ott_raw`
                WHERE 
                    ProgrammeViewingDuration >= 120
                    AND DATE(streamingSessionStart) BETWEEN STB_Max_Date() - 29 AND STB_Max_Date()
            )
        )
        GROUP BY Titles_Watched, Territory
        ORDER BY Titles_Watched
    )

    SELECT 
        Titles_Watched, UK, DE, IT
    FROM(
        SELECT *
        FROM Setup
    ) AS p 
    PIVOT(
        SUM(MAAs) FOR Territory IN('UK', 'DE', 'IT')
    ) p2
    ORDER BY Titles_Watched
    """,
project_id=project,
credentials=credentials)

Downloading: 100%|███████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 61.04rows/s]


In [16]:
# OTT_Category_Reach = pdq.read_gbq(
#     """
#     -- Find the last date in the STB table to use to find trailing 30 days
#     CREATE TEMP FUNCTION STB_Max_Date()
#     AS(
#         (SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
#         FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
#         WHERE Territory IN('GB', 'IE'))
#     );

#     SELECT 
#         c.Category, 
#         COUNT(DISTINCT(ProfileId)) MAAs
#     FROM `nbcu-ds-sandbox-a-001.wes_sandbox.OTT_With_Titles` o 
#     LEFT JOIN `nbcu-ds-sandbox-a-001.wes_sandbox.ContentTitleCategories` c
#     ON o.OriginalContentTitle = c.ContentTitle AND o.Territory = c.Territory
#     WHERE 
#         ProgrammeViewingDuration >= 120
#         AND DATE(streamingSessionStart) BETWEEN STB_Max_Date() - 29 AND STB_Max_Date()
#         AND o.Territory = 'UK'
#     GROUP BY Category
#     -- ORDER BY Category
#     UNION ALL
#     SELECT 
#         CASE WHEN c.Category LIKE 'Library%' THEN 'Library' ELSE c.Category END Category, 
#         COUNT(DISTINCT(ProfileId)) MAAs
#     FROM `nbcu-ds-sandbox-a-001.wes_sandbox.OTT_With_Titles` o 
#     LEFT JOIN (SELECT * FROM `nbcu-ds-sandbox-a-001.wes_sandbox.ContentTitleCategories` WHERE Category LIKE 'Library%') c
#     ON o.OriginalContentTitle = c.ContentTitle AND o.Territory = c.Territory
#     WHERE 
#         ProgrammeViewingDuration >= 120
#         AND DATE(streamingSessionStart) BETWEEN STB_Max_Date() - 29 AND STB_Max_Date()
#         AND o.Territory = 'UK'
#         AND c.Category != ''
#     GROUP BY Category
#     ORDER BY Category;
#     """,
# project_id=project,
# credentials=credentials)

In [17]:
Acquisitions_Last_7_Days = pdq.read_gbq(
    """
    -- Find the last date in the STB table to use to find trailing 30 days
    CREATE TEMP FUNCTION STB_Max_Date()
    AS(
        (SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE Territory IN('GB', 'IE'))
    );

    SELECT ContentTitle, COUNT(DISTINCT(ProfileId)) MAAs
    FROM(
      SELECT 
        *,
        RANK() OVER(PARTITION BY ProfileId ORDER BY streamingSessionStart ASC) First_Show
      FROM `nbcu-ds-sandbox-a-001.wes_sandbox.OTT_With_Titles`
      WHERE ProgrammeViewingDuration >= 120
    )
    WHERE 
      First_Show = 1
      AND Watch_Date BETWEEN STB_Max_Date() - 6 AND STB_Max_Date()
      AND Territory = 'UK'
    GROUP BY ContentTitle
    ORDER BY MAAs DESC
        """,
project_id=project,
credentials=credentials)

Downloading: 100%|████████████████████████████████████████████████████████████████| 183/183 [00:00<00:00, 566.17rows/s]


In [18]:
Acquisitions_Last_30_Days = pdq.read_gbq(
    """
    -- Find the last date in the STB table to use to find trailing 30 days
    CREATE TEMP FUNCTION STB_Max_Date()
    AS(
        (SELECT MAX(PARSE_DATE('%Y%m%d', pdl_event_date)) - 1
        FROM `nbcu-ds-prod-001.PeacockOnSkyDataMart.pos_stb_raw`
        WHERE Territory IN('GB', 'IE'))
    );

    SELECT ContentTitle, COUNT(DISTINCT(ProfileId)) MAAs
    FROM(
      SELECT 
        *,
        RANK() OVER(PARTITION BY ProfileId ORDER BY streamingSessionStart ASC) First_Show
      FROM `nbcu-ds-sandbox-a-001.wes_sandbox.OTT_With_Titles`
      WHERE ProgrammeViewingDuration >= 120
    )
    WHERE 
      First_Show = 1
      AND Watch_Date BETWEEN STB_Max_Date() - 29 AND STB_Max_Date()
      AND Territory = 'UK'
    GROUP BY ContentTitle
    ORDER BY MAAs DESC
        """,
project_id=project,
credentials=credentials)

Downloading: 100%|████████████████████████████████████████████████████████████████| 196/196 [00:00<00:00, 523.18rows/s]


In [19]:
Gross_Adds_Analysis = pdq.read_gbq(
    """
    SELECT *
    FROM `nbcu-ds-sandbox-a-001.wes_sandbox.TableauintlGrossAddsLapsesReactivations`
    ORDER BY WeekEnd
    """,
project_id=project,
credentials=credentials)

Downloading: 100%|████████████████████████████████████████████████████████████████| 168/168 [00:00<00:00, 502.64rows/s]


In [20]:
with pd.ExcelWriter(namefile) as filename:
    MAAs_Territory_Provider.to_excel(filename, sheet_name = 'MAAs by Territory and Provider', index = False)
    Daily_AAs_STB.to_excel(filename, sheet_name = 'Daily AAs STB', index = False)
    Daily_AAs_OTT.to_excel(filename, sheet_name = 'Daily AAs OTT', index = False)
    Daily_AAs_Starts_Usage.to_excel(filename, sheet_name = 'Daily AAs, Starts, Usage', index = False)
    First7_Analysis.to_excel(filename, sheet_name = 'First 7 Territory Analysis', index = False)
    OTT_MAAs_Days_Watched.to_excel(filename, sheet_name = '# of Days Used PoS', index = False)
    OTT_MAAs_Titles_Watched.to_excel(filename, sheet_name = '# of Titles Watched on PoS', index = False)
#     OTT_Category_Reach.to_excel(filename, sheet_name='Category Reach', index = False)
    Acquisitions_Last_7_Days.to_excel(filename, sheet_name = 'Acquisitions Last 7 Days', index = False)
#     Acquisitions_Last_30_Days.to_excel(filename, sheet_name = 'Acquisitions Last 30 Days', index = False)
    Gross_Adds_Analysis.to_excel(filename, sheet_name='Gross Adds Analysis', index = False)

In [21]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'Shob.Kakkar@nbcuni.com'
mail.CC = 'Alejandra.Zannier@nbcuni.com; darragh.mulligan@nbcuni.com'
mail.Subject = 'Weekly Data Request ' + date
mail.Body = 'print(MAAs_Territory_Provider)'
mail.HTMLBody = (
    '''<body>
        <p>
            Hi All,
            <br><br>Please find this week's data request attached. 
            <br><br>Please let me know if you have any questions.
            <br><br>MAAs ''' + date_range + ':' + MAAs_Territory_Provider.to_html(index = False) +
            '''-Wes D.
        </p>
    </body>'''
    ).format(MAAs_Territory_Provider.to_string) #this field is optional
attachment  = 'C:/Users/206707903/Documents/Python/Shob_Request/' + namefile
mail.Attachments.Add(attachment)
mail.Save()
# mail.Send()